In [ ]:
# # install packages
# !pip install matpower matpowercaseframes --quiet

In [ ]:
import matpower

print(f"matpower version: {matpower.__version__}")

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from matpowercaseframes import CaseFrames

from matpower import path_matpower

In [ ]:
path = os.path.join(path_matpower, "data/case9.m")
cf = CaseFrames(path)
cf.gencost

In [ ]:
pmin_pmax = cf.gen.loc[:, ["PMIN", "PMAX"]]
pmin_pmax

In [ ]:
# this logic of plot only support quadratic
n_gen = len(cf.gen.index)
x = [None] * n_gen
y = [None] * n_gen
for i in range(n_gen):
    pmin, pmax = cf.gen["PMIN"].iloc[i], cf.gen["PMAX"].iloc[i]
    coeff = cf.gencost[["C0", "C1", "C2"]].iloc[i].values
    x[i] = np.linspace(pmin, pmax, 100, endpoint=True)
    y[i] = [np.polyval(coeff, x_) for x_ in x[i]]
    plt.plot(x[i], y[i], label=f"gen {i}")
    plt.legend(loc="best")
plt.axis([0, None, 0, None])
plt.title("Fuel Cost Plot ($)")
plt.show()

In [ ]:
# this logic of plot only support quadratic
n_gen = len(cf.gen.index)
x = [None] * n_gen
y = [None] * n_gen
for i in range(n_gen):
    pmin, pmax = cf.gen["PMIN"].iloc[i], 50
    coeff = cf.gencost[["C0", "C1", "C2"]].iloc[i].values
    x[i] = np.linspace(pmin, pmax, 100, endpoint=True)
    y[i] = [np.polyval(coeff, x_) for x_ in x[i]]
    plt.plot(x[i], y[i], label=f"gen {i}")
    plt.legend(loc="best")
plt.axis([0, None, 0, None])
plt.title("Fuel Cost Plot Up To 50 MWh ($)")
plt.show()

In [ ]:
df_components_pmin = cf.gencost[["C2", "C1", "C0"]].copy()
df_components_pmin.columns = ["A", "B", "C"]
df_components_pmin.loc[:, "A"] = (
    df_components_pmin["A"].values * cf.gen["PMIN"].values * cf.gen["PMIN"].values
)
df_components_pmin.loc[:, "B"] = df_components_pmin["B"].values * cf.gen["PMIN"].values
df_components_pmin

In [ ]:
df_components_pmax = cf.gencost[["C2", "C1", "C0"]].copy()
df_components_pmax.columns = ["A", "B", "C"]
df_components_pmax.loc[:, "A"] = (
    df_components_pmax["A"].values * cf.gen["PMAX"].values * cf.gen["PMAX"].values
)
df_components_pmax.loc[:, "B"] = df_components_pmax["B"].values * cf.gen["PMAX"].values
df_components_pmax